# CS310 Natural Language Processing
# Lab 2: Neural Text Classification

In this lab, we will practice building a neural network for text classification. 

The tutorial code is adopted from the official PyTorch tutorial: *Text classification with the torchtext library*

https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html#text-classification-with-the-torchtext-library

## Install datasets

Url: https://huggingface.co/docs/datasets/en/installation
```bash
conda install -c huggingface -c conda-forge datasets
```

In [87]:
import torch
from torch.utils.data.dataset import random_split
from data_utils import DatasetIterator, get_tokenizer, build_vocab_from_iter, to_map_style_dataset

## Load data

In [88]:
# Test datasets package, with SST2
from datasets import load_dataset

ds = load_dataset('glue', 'sst2') # it take a while

In [89]:
# Check the raw data
train_iter = DatasetIterator(ds['train'])

count = 0
for item in train_iter:
    print(item)
    count += 1
    if count > 5:
        break

('hide new secretions from the parental units ', 0)
('contains no wit , only labored gags ', 0)
('that loves its characters and communicates something rather beautiful about human nature ', 1)
('remains utterly satisfied to remain the same throughout ', 0)
('on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 0)
("that 's far too tragic to merit such superficial treatment ", 0)


## Apply Tokenizer

In [90]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [91]:
# Check the output of yield_tokens()
count = 0
for tokens in yield_tokens(DatasetIterator(ds['train'])): # Use a new iterator
    print(tokens)
    count += 1
    if count > 3:
        break

['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units']
['contains', 'no', 'wit', ',', 'only', 'labored', 'gags']
['that', 'loves', 'its', 'characters', 'and', 'communicates', 'something', 'rather', 'beautiful', 'about', 'human', 'nature']
['remains', 'utterly', 'satisfied', 'to', 'remain', 'the', 'same', 'throughout']


## Build Vocabulary

First, we build the vocabulary using the `build_vocab_from_iterator` function.

In [92]:
vocab = build_vocab_from_iter(yield_tokens(DatasetIterator(ds['train'])), specials=["<unk>"])

You can see that all strings are converted into integer IDs.

In [93]:
# Check the vocab
print(vocab(['here', 'is', 'an', 'example']))
print(vocab(['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units']))
print(vocab(['of', 'saucy']))

# What about unknown words, i.e., out-of-vocabulary (OOV) words?
print(vocab(['here', 'is', 'a', '@#$@!#$%']))

[224, 10, 16, 1587]
[4460, 92, 12405, 38, 1, 7259, 8990]
[5, 6549]
[224, 10, 3, 0]


Next, further define the `text_pipeline` and `label_pipeline` functions, for converting strings to integers. 

In [94]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

# Test text_pipeline()
tokens = text_pipeline('here is an example')
print(tokens)

# Test label_pipeline()
lbl = label_pipeline('1')
print(lbl)

[224, 10, 16, 1587]
1


## T1. Batchify Data 

Your goal is to define the `Collate_batch` function, which will be used to process the "raw" data batch.

*Hint*: In the loop of `collate_batch`, the labels, tokens, and the offsets of all the examples are collected into three lists. Finally, the lists are converted into tensors. 

*Hint*: The `offsets` need to contain the cumulative positions of tokens in the batch. 
For example, if the batch contains 3 examples, whose lengths are `[1,3,2]`, then the final offsets should be `[0,1,4,6]`.

In [95]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, token_ids_list, offsets = [], [], [0]
    for _text, _label in batch:
        label_list.append(label_pipeline(_label))
        token_ids = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        token_ids_list.append(token_ids)
        offsets.append(token_ids.size(0)) # Note that offsets contains the length (number of tokens) of each example

    ### START YOUR CODE ###
    # 将标签列表转换为张量
    labels = torch.tensor(label_list, dtype=torch.int64)
    # 将所有token_ids拼接成一个大的tensor
    token_ids = torch.cat(token_ids_list, dim=0)
    # 计算偏移量的累积值
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # 用cumsum计算累积偏移量
    ### END YOUR CODE ###

    return labels.to(device), token_ids.to(device), offsets.to(device)

Next, use the defined `collate_batch` function to create the dataloader.

In [96]:
# Use collate_batch to generate the dataloader
train_iter = DatasetIterator(ds['train'])

dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [97]:
# Test the dataloader
for i, (labels, token_ids, offsets) in enumerate(dataloader):
    if i == 0:
        break

# What does offsets mean?
print('Number of tokens in this batch: ', token_ids.size(0))
print('Number of examples in one batch: ', labels.size(0))
print('Example 0: ', token_ids[offsets[0]:offsets[1]])
print('Example 7: ', token_ids[offsets[7]:])

# You are expected to see the following output:
# Number of tokens in this batch:  82
# Number of examples in one batch:  8
# Example 0:  tensor([ 4460,    92, 12405,    38,     1,  7259,  8990])
# Example 7:  tensor([   5, 6549])

Number of tokens in this batch:  82
Number of examples in one batch:  8
Example 0:  tensor([ 4460,    92, 12405,    38,     1,  7259,  8990])
Example 7:  tensor([   5, 6549])


## T2. Define the Model

The model consists of two parts of parameters: an embedding layer and a fully connected layer.

Your need to first initialize an `nn.EmbeddingBag` instance and a `nn.Linear` instance:
- The embedding layer should be initialized with `vocab_size`, `embed_dim`, and `sparse=False`.
- The fully connected layer should have `embed_dim` as input size and `num_class` as output size.

Then, in the `forward` function, the embedding layer should called with `token_ids` and `offsets` as inputs. The output of embedding layer is fed to the fully connected layer to get the final output. 

In [98]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        ### START YOUR CODE ###
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)  # 初始化EmbeddingBag层
        self.fc = nn.Linear(embed_dim, num_class)  # 初始化全连接层
        ### END YOUR CODE ###
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, token_ids, offsets):
        ### START YOUR CODE ###
        embedded = self.embedding(token_ids, offsets)  # 使用embedding层
        out = self.fc(embedded)  # 将嵌入输出传给全连接层
        ### END YOUR CODE ###
        return out

In [99]:
# Build the model
train_iter = DatasetIterator(ds['train'])
num_class = len(set([label for (_, label) in train_iter]))
vocab_size = len(vocab)
emsize = 64 # embedding size
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [100]:
# Test the model
model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(dataloader):
        output = model(token_ids, offsets)
        if i == 0:
            break

print('output size:', output.size())

# You are expected to see the following output:
# output size: torch.Size([8, 2])

output size: torch.Size([8, 2])


## T3. Define the loss function

Cross entropy loss should be used. You can use `torch.nn.CrossEntropyLoss` to define the loss function.

In [101]:
# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 8  # batch size for training

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

Test if the loss function `criterion` works

In the cell below, implement the **manual computation** of cross entropy loss. Use the formula $-y_i\log(\hat{y_i})$, 

where $y_i$ is the $i$-th ground truth label in `labels`, and $\hat{y_i}$ is the predicted probability in `output` of the correponding label.

In [102]:
# First, obtain some output and labels
model.eval()
with torch.no_grad():
    for i, (labels, token_ids, offsets) in enumerate(dataloader):
        output = model(token_ids, offsets)
        print(f"batch {i} output: {output}")
        if i == 0:
            break
print('output shape:', output.shape)



loss = criterion(output, labels)
print('loss:', loss)

# Manually calculate the loss
loss_manual = []
for i in range(output.shape[0]):
    probs = torch.nn.functional.softmax(output[i], dim=-1)  # 使用softmax计算预测的概率
    # 获取对应标签的概率
    correct_prob = probs[labels[i]]
    # 使用交叉熵公式计算损失
    l = -torch.log(correct_prob)
    loss_manual.append(l)
loss_manual = torch.stack(loss_manual)
print('loss_manual mean:', loss_manual.mean())


batch 0 output: tensor([[ 0.0618, -0.0895],
        [-0.1845, -0.1673],
        [ 0.1175,  0.1440],
        [ 0.0052, -0.3349],
        [ 0.1299, -0.3681],
        [-0.1505, -0.0839],
        [ 0.0480,  0.1447],
        [-0.6846,  0.6014]])
output shape: torch.Size([8, 2])
loss: tensor(0.5789)
loss_manual mean: tensor(0.5789)


## T4. Train and Evaluate Functions

Define train and evaluate functions.

You need to implement the forward pass, loss computation, backward propagation, and parameter update in the `train` function. 

Also, for each batch of data, calculate the total number of correctly predicted examples, by comparing `output` and `labels`.

In [103]:
import time

def train(model, dataloader, optimizer, criterion, epoch: int):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (labels, token_ids, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        ### START YOUR CODE ###
        # Forward pass
        output = model(token_ids, offsets)
        ### END YOUR CODE ###
        try:
            ### START YOUR CODE ###
            # Compute loss
            loss = criterion(output, labels)
            ### END YOUR CODE ###
        except Exception:
            print('Error in loss calculation')
            print('output: ', output.size())
            print('labels: ', labels.size())
            print('token_ids: ', token_ids)
            print('offsets: ', offsets)
            raise
        ### START YOUR CODE ###
        # Backward propagation, grad clipping, and optimization
        loss.backward()  # 反向传播
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)  # 梯度裁剪，防止梯度爆炸
        optimizer.step()  # 更新参数
        ### END YOUR CODE ###

        ### START YOUR CODE ###
        # Calculate correct prediction in current batch
        _, predicted = output.max(1)  # 获取预测的类别
        total_acc += (predicted == labels).sum().item()  # 计算正确的预测数量并累加
        ### END YOUR CODE ###

        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

@torch.no_grad()
def evaluate(model, dataloader, criterion):
    model.eval()
    total_acc, total_count = 0, 0

    for idx, (label, text, offsets) in enumerate(dataloader):
        ### START YOUR CODE ###
        # Similar to the code in train function, but without backpropagation
        # 前向传播，计算输出
        output = model(text, offsets)
        # 计算损失
        loss = criterion(output, label)
        ### END YOUR CODE ###
        
        # 计算正确的预测数量
        _, predicted = output.max(1)
        total_acc += (predicted == label).sum().item()
        ### END YOUR CODE ###
        total_count += label.size(0)

    return total_acc / total_count

## Load train, valid, and test data

In [104]:
# Prepare train, valid, and test data
train_iter = DatasetIterator(ds['train'])
test_iter = DatasetIterator(ds['test'])
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

## Main Training Loop

In [105]:
# Run the training loop
total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    train(model, train_dataloader, optimizer, criterion, epoch)
    accu_val = evaluate(model, valid_dataloader, criterion)

    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val

    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

| epoch   1 |   500/ 7998 batches | accuracy    0.592
| epoch   1 |  1000/ 7998 batches | accuracy    0.665
| epoch   1 |  1500/ 7998 batches | accuracy    0.721
| epoch   1 |  2000/ 7998 batches | accuracy    0.741
| epoch   1 |  2500/ 7998 batches | accuracy    0.759
| epoch   1 |  3000/ 7998 batches | accuracy    0.779
| epoch   1 |  3500/ 7998 batches | accuracy    0.800
| epoch   1 |  4000/ 7998 batches | accuracy    0.805
| epoch   1 |  4500/ 7998 batches | accuracy    0.812
| epoch   1 |  5000/ 7998 batches | accuracy    0.809
| epoch   1 |  5500/ 7998 batches | accuracy    0.822
| epoch   1 |  6000/ 7998 batches | accuracy    0.828
| epoch   1 |  6500/ 7998 batches | accuracy    0.830
| epoch   1 |  7000/ 7998 batches | accuracy    0.838
| epoch   1 |  7500/ 7998 batches | accuracy    0.837
-----------------------------------------------------------
| end of epoch   1 | time: 24.07s | valid accuracy    0.856 
-----------------------------------------------------------
| epoch  

In [106]:
# Save the model
torch.save(model.state_dict(), "text_classification_model.pth")

## Evaluate with Test Data

This is a necessary step. But since the `test` split of SST2 is not annotated, we will use the `dev` split here to pretend it is the test data.

In [107]:
accu_test = evaluate(model, valid_dataloader, criterion)
print("test accuracy {:8.3f}".format(accu_test))

# Your test accuracy should be around 0.9

test accuracy    0.916


## Predict

Test the model with a few unannotated examples.

In [108]:
sentiment_labels = ['negative', 'positive']

def predict(text, model, vocab, tokenizer, labels):
    model.eval()
    with torch.no_grad():
        text = torch.tensor(vocab(tokenizer(text)), device=device)
        output = model(text, torch.tensor([0], device=device))
        return labels[output.argmax(1).item()]

ex_text_str = "A very well-made, funny and entertaining picture."
print("This is a %s sentiment." % (predict(ex_text_str, model, vocab, tokenizer, sentiment_labels)))

This is a positive sentiment.


Congratulations! You have successfully built and trained a neural network model to classify sentiment in text data.